In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import molsysmt as msm

In [3]:
pdbs_to_be_tested = ["1BG3", "2PGI", "4PFK", "4ALD", "3YPI", "3GPD",
                    "13PK", "1BQ3", "7ENL", "1A3W"]

In [4]:
molsys = msm.convert('1BG3')

/home/diego/projects@dprada/MolSysMT/molsysmt/item/mmtf_MMTFDecoder/to_molsysmt_Topology.py:32: UserWarning: The structure in the PDB has biological assemblies. There are geometrical transformations proposed in the structure. See the following issue in the source code repository: https://github.com/uibcdf/MolSysMT/issues/33
  warnings.warn(warning_message)
/home/diego/projects@dprada/MolSysMT/molsysmt/item/mmtf_MMTFDecoder/to_molsysmt_Topology.py:46: UserWarning: The bioassembly has a different number of chains than the total amount of chains
  warnings.warn(warning_message)


In [5]:
molsys = msm.build.remove_atoms_with_alternate_locations(molsys)

In [6]:
msm.view(molsys)

NGLWidget()